# Jonathan Halverson
# Wednesday, February 8, 2017
# Latent Semantic Analysis of Wikipedia Biographies

Here we download the biographies of nine people (3 scientists, 3 filmmakers and 3 athletes). We then clean and tokenize the text of each before applying TF-IDF. These vectors are then transformed using LSI with three topics. Each person is correctly assigned to their class.

The stop words were downloaded using nltk.download() and then moved to /usr/local/share/. Note that all of Wikipedia in English is about 8 GB.

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
def scrape_and_tokenize(person):
    # download and parse the biography
    base_url = 'https://en.wikipedia.org/wiki/'
    r = requests.get(base_url + person)
    soup = BeautifulSoup(r.content, 'lxml')
    
    # extract the text of each paragraph
    raw_text = ''
    for paragraph in soup.find_all('p'):
        raw_text += paragraph.get_text()
    
    # keep only alphabetical characters and split on whitespace
    letters_only = re.sub("[^a-zA-Z]", " ", raw_text)
    words = letters_only.lower().split()
    
    # count the words and filter based on count and stopwords, apply stemming
    count = Counter(words)
    porter = PorterStemmer()
    stops = stopwords.words("english")
    words = [porter.stem(word) for word in words if (word not in stops) and (count[word] > 1) and (len(word) > 1)]
    return words

Create a list of words from each biography:

In [3]:
einstein = scrape_and_tokenize('Albert_Einstein')
newton = scrape_and_tokenize('Isaac_Newton')
darwin = scrape_and_tokenize('Charles_Darwin')
spielberg = scrape_and_tokenize('Steven_Spielberg')
allen = scrape_and_tokenize('Woody_Allen')
cameron = scrape_and_tokenize('James_Cameron')
jordan = scrape_and_tokenize('Michael_Jordan')
brady = scrape_and_tokenize('Tom_Brady')
williams = scrape_and_tokenize('Serena_Williams')

Note that the documents are the raw files as a single string. The texts are a list of words for each document.

In [4]:
from gensim import corpora, models

texts = [einstein, newton, darwin, spielberg, allen, cameron, jordan, brady, williams]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Pass the bag-of-words vectors to the TF-IDF transformer:

In [5]:
tfidf = models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]

Latent semantic analysis (LSA) is a technique in natural language processing, in particular distributional semantics, of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. LSA assumes that words that are close in meaning will occur in similar pieces of text. A matrix containing word counts per paragraph (rows represent unique words and columns represent each paragraph) is constructed from a large piece of text and a mathematical technique called singular value decomposition (SVD) is used to reduce the number of rows while preserving the similarity structure among columns. Words are then compared by taking the cosine of the angle between the two vectors (or the dot product between the normalizations of the two vectors) formed by any two rows. Values close to 1 represent very similar words while values close to 0 represent very dissimilar words.

Transform the TF-IDF vectors using LSI with two topics:

In [6]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)
corpus_lsi = lsi[corpus_tfidf]

Let's examine the results:

In [7]:
for doc in corpus_lsi:
    print(doc)

[(0, -0.24859894523288342), (1, 0.1209746499255403), (2, -0.58331347415145907)]
[(0, -0.18070874390178085), (1, 0.07910880806567748), (2, -0.57572366442497569)]
[(0, -0.21411807231770516), (1, 0.082987071545228372), (2, -0.4700030997317089)]
[(0, -0.54416791686914689), (1, 0.26389216475435484), (2, 0.23963274882414004)]
[(0, -0.46593754033607016), (1, 0.2555211285893027), (2, 0.20192557766247804)]
[(0, -0.44655385752422228), (1, 0.3026957586455058), (2, 0.24337917306696857)]
[(0, -0.29906397337808216), (1, -0.53833600413431137), (2, 0.045503767091347201)]
[(0, -0.28134484894429879), (1, -0.58949089234474028), (2, 0.022923400355726965)]
[(0, -0.31930533900753194), (1, -0.41690571115979808), (2, -0.011084734658368893)]


We see that the method correctly assigns the people to their class. Negative signs appear because cosine similarit is used.

In [8]:
# print the most contributing words (both positively and negatively) for each of the first ten topics
lsi.print_topics(num_words=5)

[(0,
  u'-0.439*"spielberg" + -0.366*"cameron" + -0.362*"allen" + -0.234*"film" + -0.233*"jordan"'),
 (1,
  u'-0.442*"jordan" + -0.411*"bradi" + 0.262*"cameron" + 0.225*"spielberg" + 0.206*"allen"'),
 (2,
  u'-0.513*"newton" + -0.485*"einstein" + -0.386*"darwin" + 0.213*"cameron" + 0.206*"spielberg"')]

It may be wise to remove the name of the person in the texts since these are the most contributing words.

Let's try two more people that aren't in the dictionary:

In [9]:
kubrick = scrape_and_tokenize('Stanley_Kubrick')
kubrick_bow = dictionary.doc2bow(kubrick)
lsi[tfidf[kubrick_bow]]

[(0, -0.1229246693577853),
 (1, 0.055600920407780217),
 (2, 0.031805002367248994)]

In [10]:
karle = scrape_and_tokenize('Jerome_Karle')
karle_bow = dictionary.doc2bow(karle)
lsi[tfidf[karle_bow]]

[(0, -0.02281424699691606),
 (1, 0.00030634142974144746),
 (2, -0.01727652909282737)]

The model correctly assigns Kubrick to the filmmaker class but fails to assign Karle to the scientist class. This is probably because he was not included in the dictionary and his specialization is being ignored. Also, his biography is far shorter than the others.

### Similarities

Here we examine the similarity between a new document and the set of previous documents using the raw corpus and the TF-IDF corpus:

In [11]:
from gensim import similarities

index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[lsi[kubrick_bow]]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(3, 0.99389589), (4, 0.99349183), (5, 0.9804979), (2, 0.24349047), (8, 0.24224375), (0, 0.22379339), (6, 0.13625595), (1, 0.11552891), (7, 0.062188789)]


In [12]:
index = similarities.MatrixSimilarity(lsi[corpus_tfidf])
sims = index[lsi[tfidf[kubrick_bow]]]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(3, 0.98907512), (4, 0.98866612), (5, 0.96864134), (2, 0.22047155), (8, 0.21588182), (0, 0.20936118), (1, 0.098405346), (6, 0.096707374), (7, 0.028004209)]


In [13]:
sims = index[lsi[tfidf[karle_bow]]]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(2, 0.87038624), (0, 0.85450494), (1, 0.80917197), (8, 0.48884705), (3, 0.44880307), (4, 0.44376296), (5, 0.3587091), (6, 0.33225757), (7, 0.3123211)]


We see that the three filmmakers were found to be most similar to Kubrick and Karle is correctly associated with the scientists.